In [29]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
pd.set_option('display.max_columns', 5400)
pd.set_option('display.max_colwidth', 500)

#input roba
df_train = pd.read_csv('processed_train.csv',index_col = 0)
df_test = pd.read_csv('processed_test.csv',index_col = 0)
df=pd.concat([df_train,df_test])
df.date = df.date.apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))



             sku  salesW-1  scope  salesW-2  rolling1  rolling5     diff1  \
dateL                                                                       
2017-01-14  2696   14863.0      0    9117.0   14863.0   12961.4   -5746.0   
2017-01-21  2696   15068.0      0   14863.0   15068.0   13155.0    -205.0   
2017-01-28  2696   15363.0      0   15068.0   15363.0   13516.8    -295.0   
2017-02-04  2696   15685.0      0   15363.0   15685.0   14019.2    -322.0   
2017-02-11  2696   15983.0      0   15685.0   15983.0   15392.4    -298.0   
2017-02-18  2696   15270.0      0   15983.0   15270.0   15473.8     713.0   
2017-02-25  2696   14503.0      0   15270.0   14503.0   15360.8     767.0   
2017-03-04  2696   14936.0      0   14503.0   14936.0   15275.4    -433.0   
2017-03-11  2696   16173.0      0   14936.0   16173.0   15373.0   -1237.0   
2017-03-18  2696   16730.0      0   16173.0   16730.0   15522.4    -557.0   
2017-03-25  2696   17052.0      0   16730.0   17052.0   15878.8    -322.0   

In [43]:
from scipy.stats import pearsonr
# df è il data frame che contiene i dati concatenati, SENZA ESSERE RIODINATI
frame=df.copy()
#DA QUI fa preprocessing per comodità mia a non riscrivere tutto
frame.rename(columns={
    'Unnamed: 0':'dateL',
    'size (GM)':'size',
    'POS_exposed w-1':'NumStoreEvidW-1',
    'volume_on_promo w-1':'percProdPromoW-1',
    'sales w-1':'salesW-1',
    'sales w-2':'salesW-2',
    'target':'salesW'},inplace=True)
frame['dateL']=pd.to_datetime(frame['date']) 
frame.set_index('dateL',inplace=True)

#definizione della funzione che calola mape
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
#elimino tutto tranne le colonne tili alla mia predizione
frame.drop(frame.columns.difference(['salesW','diff1','salesW-1','salesW-2','rolling1','rolling5', 'sku','scope']), 1, inplace=True)
frame=frame.dropna()


mapeTot,elem=0,0
#listone serve per contenere tutti gli sku
listone=list(dict.fromkeys(frame['sku']))#listone contiene i singoli sku



for i in listone:#questo ciclo su i è il ciclo madre, ogni iterazione fa uno sku del test set e produce un mape
    pf=frame[frame['sku']==i]
    if pf.iloc[0]['scope']==0:#scarto train set, senza peggiora
        continue  
    #uso una variabile di supporto, questo è il dataframe su cui opero aggiungendo colonne di prodotti correlati 
    df_stock = pd.DataFrame(index=pf.index)
    df_stock[['diff1','salesW-1','salesW-2','rolling1','rolling5']]=pf[['diff1','salesW-1','salesW-2','rolling1','rolling5']]
    df_stock['t'] = pf['salesW']
    df_stock.head()

  
    for j in listone:#qui per ogni sku controllo tutti gli altri se sono correlati o no
        if (j==i) | (frame.iloc[0]['sku']==i) :#scarto duplicati inutili e train set che fa peggiorare predizione
            continue
        curr=frame[frame['sku']==j]
        if curr.iloc[0]['scope']==0:
            continue
            
        pearsonCorrSales,_=pearsonr(pf['salesW'],curr['salesW'])#calcola correlazione di pearson
        if pearsonCorrSales>0.6  :#se ho alta correlazione aggiungo informazioni di prodotti correlati
            df_stock['prod'+str(frame.iloc[0]['sku'])]=curr['salesW']
            df_stock['prod'+str(frame.iloc[0]['sku'])+'W-1']=curr['salesW-1']
            df_stock['prod'+str(frame.iloc[0]['sku'])+'W-2']=curr['salesW'].shift(2)
                                
    #-------------------------------------------
    # eliminate the trailing NaN
    df_stock.dropna(inplace=True)
    #divido in train e test set
    target_variable = 't'
    input_variables = df_stock.columns[df_stock.columns!=target_variable]

    X=df_stock[input_variables]
    y=df_stock[target_variable]
    
    X_train, X_test = X[:'2019-6-29'], X['2019-6-29':]
    y_train, y_test = y[:'2019-6-29'], y['2019-6-29':]

    lr_model = LinearRegression()
    lr_model.fit(X_train,y_train)

    yt = lr_model.predict(X_train)
    yp = lr_model.predict(X_test)
    #calcola rmse e mape per ogni sku
    print('RMSE on Test %.3f'%math.sqrt(mean_squared_error(yp,y_test)))
    print("mape del prod con sku: "+str(pf.iloc[0]['sku'])+" : "+str(  np.mean(np.abs(((y_test-yp)/y_test))))  )
    mapeTot=mapeTot+mape(y_test,yp)
    elem=elem+1#tieni traccia di numero elementi
    
#calcola mape totale
print('MAPE totale %.3f'%(mapeTot/elem))



RMSE on Test 5644.236
mape del prod con sku: 1027.0 : 0.05492149825917594
RMSE on Test 3423.307
mape del prod con sku: 1035.0 : 0.05355105040706617
RMSE on Test 3080.270
mape del prod con sku: 144.0 : 0.07519353010036163
RMSE on Test 2312.018
mape del prod con sku: 1051.0 : 0.07238115259819423
RMSE on Test 4322.717
mape del prod con sku: 546.0 : 0.04214577602211151
RMSE on Test 1819.817
mape del prod con sku: 1058.0 : 0.026607116032427634
RMSE on Test 3311.762
mape del prod con sku: 549.0 : 0.04959601299633074
RMSE on Test 4388.260
mape del prod con sku: 1065.0 : 0.03043561818113233
RMSE on Test 5494.976
mape del prod con sku: 554.0 : 0.025292008122016988
RMSE on Test 4347.923
mape del prod con sku: 686.0 : 0.08015089858298009
RMSE on Test 3259.491
mape del prod con sku: 688.0 : 0.0564538857782316
RMSE on Test 3659.336
mape del prod con sku: 1206.0 : 0.05496355764724018
MAPE totale 5.181
